<center><img src="images/header.png"></center>

<h1><center>Алгоритмы интеллектуальной обработки больших объемов данных</center></h1>
<hr>
<h2><center>Выявление выбросов и аномалий (практика)</center></h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

# Простой детектор аномалий на временных рядах

**z-score detector**

Гипермараметр: ширина окна.

Вычиление:
$$ z(x_t)=\left|\frac{x_t−\bar{x}_t}{s_t}\right| $$

где
$$ x_t=(1/w)\sum_{i=t−w}^{t-1} x_i $$

$$ s_t=(1/w)\sum_{i=t−w}^{t-1} (x_i - \bar{x}_t)^2 $$

Чем выше скор - тем больше шанс того, что наблюдение аномально

Имплементируйте скор для ряда `nyc_taxi.tsv`. Когда мера работать не будет?

# Данные о диетах в разных странах

In [ ]:
df_diet = pd.read_csv('data/diet.csv', sep=';').set_index('Countries').iloc[:, :-1]

# Данные по фроду

Воспользуемся данными из https://www.kaggle.com/dalpozz/creditcardfraud/data

**Time** - the seconds elapsed between each transaction and the first transaction in the dataset. Numeric  
**V1** - First principle component Numeric  
**V2** - Second principle component Numeric  
**V3** - Third principle component Numeric  
...  
**V28** - Twenty-eighth principle component Numeric  
**Amount** - Transaction Amount Numeric   
**Class** - The actual classification classes Numeric  

The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on http://mlg.ulb.ac.be/BruFence and http://mlg.ulb.ac.be/ARTML

Please cite: Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

In [ ]:
df = pd.read_csv('data/creditcard.csv.zip')

In [ ]:
df.head()

Во-первых, обратим внимание, что у нас есть шкала времение. Из-за этого может возникнуть нюансы, но мы на них пока не будем обращать внимания. Использовать Time в качестве признака, конечно-же не стоит..

Так же обратим внимание, что классы у нас очень несбалансированные. Отсюда возникает несколько вопросов:
* Какой **метод** использовать?
* Как мерить **качество**?

Насчет первого - будем использовать специализированные для этого метод - **изолирующий лес**. 

Насчет второго - одназначного ответа, увы **нет**. Давайте будем измерять несколькими способами и обсудим что в них плохо или хорошо. Кстати, в подобных задачах хочется получать не просто "вероятность" фрода, а саму разметку...

FYI можете посчитать интересную статью про меры качества в МО и из экономическую целесообразность - http://blog.mldb.ai/blog/posts/2016/01/ml-meets-economics/

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import auc, roc_curve, matthews_corrcoef, roc_auc_score
from sklearn.metrics import precision_recall_curve, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit

Среди импортированных метрик вы, возможно, встретили ранее невиданную: коэффициент корреляции Мэтьюса:
$$\text{MCC}=\frac{TP\times TN-FP\times FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}}$$

По-большому счету - это просто коэффициент корреляции между двумя бинарными векторами. Но различные эмпирические исследования показали, что он хорошо себя проявляет в задачах с сильным дисбалансом классов.

## Подготовка данных

* Выделим тестовое множество, как последние 20% данных
* Выделим обучающие, тестовые и валидационные матрицы и разметки 

## Изолирующий лес

In [ ]:
iso = IsolationForest(...)

### Мониторинг работы алгоритма

Отберем несколько объектов с обоих классов для мониторинга

Разработаем следующий пайплайн:
* Будут перебираться 2 параметра - кол-во сэмплов и кол-во деревьев
* Для каждой настойки параметра будем записывать *скор* аномальности
* Попробуем прикинуть выходят ли скоры на стабильную траекторию?